# File Search

## Install Packages

In [ ]:
%pip install azure-ai-projects==1.0.0b12
%pip install azure-identity
%pip install azure-ai-agents==1.1.0b3

## Import the libraries

In [3]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import (
    FilePurpose,
    FileSearchTool,
    ListSortOrder,
    RunAdditionalFieldList,
    RunStepFileSearchToolCall,
    RunStepToolCallDetails,
)

## Create the AI Project Client

NOTE: 
- If you don't have Azure CLI, you first need to install it:
   curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
- Login to Azure
   az login



In [6]:
# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]  # Ensure the PROJECT_ENDPOINT environment variable is set

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

## Setup the file search tool

In [ ]:
import os

file_directory = "../data/healthplans"

# List all files in the directory
try:
    filenames = os.listdir(file_directory)
    print(filenames)
except FileNotFoundError:
    print(f"Directory '{file_directory}' not found.")

# Get the full path of a file
def get_filepath_for_filename(filename: str) -> str:
    base_directory = file_directory
    return os.path.join(base_directory, filename)

# Upload files to the project client
file_ids: list[str] = []
for path in [get_filepath_for_filename(filename) for filename in filenames]:
    with open(path, "rb") as file:
        file = project_client.agents.files.upload_and_poll(file_path=path, purpose=FilePurpose.AGENTS)
        print(f"Uploaded file, file ID: {file.id}")
        file_ids.append(file.id)

# Create a vector store with the uploaded files
vector_store = project_client.agents.vector_stores.create_and_poll(file_ids=file_ids, name="my_vectorstore")
print(f"Created vector store, vector store ID: {vector_store.id}")

['Northwind_Standard_Benefits_Details.pdf', 'Northwind_Health_Plus_Benefits_Details.pdf']
Uploaded file, file ID: assistant-1s6N6QkaK6eeP5tFvGaRRM
Uploaded file, file ID: assistant-T49j1LTf5uieyYRZ8ooMH9
Created vector store, vector store ID: vs_d9Gg6v4oFzmCfCrDOyoGDJB1


## Create the agent

In [9]:
# Create a file search tool
file_search = FileSearchTool(vector_store_ids=[vector_store.id])

# Create an agent with the file search tool
agent = project_client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
    name="my-filesearch-agent",  # Name of the agent
    instructions="You are a helpful agent and can search information from uploaded files",  # Instructions for the agent
    tools=file_search.definitions,  # Tools available to the agent
    tool_resources=file_search.resources,  # Resources for the tools
)
print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_uuh0QG6brtVb8i128vrT04kB


## Create thread

In [10]:
# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_FgFCEklWZsCT94hrkk7w0Kbu


## Helper function
- adds messages to the thread
- run the agent
- display the agent response

In [11]:
def run_agent(user_input):
    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",  # Role of the message sender
        content=user_input,  # Message content
    )
    print(f"Created message, ID: {message['id']}")

     # Create and process agent run in thread with tools
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    elif run.status == "completed":
        # Fetch the first message in the thread to see the agent's response
        messages = list(project_client.agents.messages.list(thread_id=thread.id))
        if messages:
            first_message = messages[0]
            content = first_message['content']
            # Extract the text value if present
            if isinstance(content, list) and content:
                text_value = content[0].get('text', {}).get('value', '')
                print(text_value)

## Run the agent

In [12]:
user_input = "Provide details on the Northwind Standard Plan"
run_agent(user_input)

Created message, ID: msg_AKGSV0b7f8x1RgJ4eJlFhR5x
Run finished with status: completed
The Northwind Standard Plan is a basic health insurance plan that offers coverage for various medical, vision, and dental services. Here are the key details:

1. **Covered Services**:
   - **Medical**: Includes primary care visits, specialist visits, preventive care, and prescription drug coverage.
   - **Vision**: Covers routine eye exams and prescription eyeglasses or contact lenses.
   - **Dental**: Covers preventive dental care such as cleanings and x-rays.

2. **Costs and Payments**:
   - **Premiums**: Monthly payments based on the age and health of the policyholder and the type of coverage chosen.
   - **Deductibles**: $2,000 per person per year. This amount must be paid out of pocket before the plan covers medical expenses.
   - **Co-payments and Co-insurance**: After meeting the deductible, co-payments are $30 for primary care visits and $50 for specialist visits. Co-insurance is 20% of the re

In [13]:
user_input = "Provide a tabular comparison between the Northwind Standard and Northwind Healthplus plans"
run_agent(user_input)

Created message, ID: msg_uzWTRkRAxRR7WVxUc06ugjsb
Run finished with status: completed
Here is a tabular comparison of the Northwind Standard and Northwind Healthplus plans based on the provided details:

| Feature/Component                   | Northwind Standard Plan                                                                             | Northwind Healthplus Plan                                                                           |
|-------------------------------------|-----------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------|
| **Covered Services**                | Medical, vision, dental, preventive care, prescription drugs                                       | Medical, vision, dental, preventive care, prescription drugs, mental health and substance abuse, emergency services |
| **Premiums**                        | Based on 

## Cleanup resources

In [ ]:
# Cleanup resources

# Delete the vector store after use
project_client.agents.vector_stores.delete(vector_store.id)
print("Deleted vector store")

# Delete the files after use
project_client.agents.files.delete(file_id=file.id)
print("Deleted file")

# Delete the agent after use
project_client.agents.delete_agent(agent.id)
print("Deleted agent")


Deleted vector store
Deleted file
